Model Training
Import Data and Required Packages
Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

Import the CSV Data as Pandas DataFrame

In [2]:
df=pd.read_csv(r"D:\Data Analyst\ML_Project\src\Notebook\student.csv")

In [3]:
# First 5 records
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# Preparing X and Y variables
X=df.drop(columns=["math score"],axis=0)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [5]:
y=df['math score']

In [6]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [7]:
# Unique Category

print("Categories in 'Gender' attribute: ", end=" ")
print(df['gender'].unique())
print("Categories in 'race_ethnicity' attribute: ", end=" ")
print(df['race/ethnicity'].unique())
print("Categories in 'parental level of education' attribute: ", end=" ")
print(df['parental level of education'].unique())
print("Categories in 'lunch' attribute: ", end=" ")
print(df['lunch'].unique())
print("Categories in 'test preparation course' attribute: ", end=" ")
print(df['test preparation course'].unique())



Categories in 'Gender' attribute:  ['female' 'male']
Categories in 'race_ethnicity' attribute:  ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental level of education' attribute:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' attribute:  ['standard' 'free/reduced']
Categories in 'test preparation course' attribute:  ['none' 'completed']


In [8]:
# Create column transform with 3 types of transformers

num_feature = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer= StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor= ColumnTransformer(
[    
    ("OneHotEncoder", oh_transformer, cat_features),
    ("StanderdScaler", numeric_transformer, num_feature),
]
)


In [9]:
X= preprocessor.fit_transform(X)

In [10]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [11]:
X.shape

(1000, 19)

In [12]:
# separate data for test and train

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [13]:
# function define all the evaluation matrices

def evaluate_model(true, predicted):
    mae= mean_absolute_error(true, predicted)
    mse= mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square= r2_score(true,predicted)
    return mae, mse, rmse, r2_square

In [14]:
models= {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor" : AdaBoostRegressor()}
model_list=[]
r2_list=[]

for i in range (len(list(models))):
    model = list(models.values()) [i]
    model.fit(X_train, y_train)
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)

